In [ ]:


import pandas as pd
import numpy as np
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import scipy.stats as stats
import itertools

import math
import itertools

import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ttest_ind

from scipy.stats import mannwhitneyu
import statsmodels.api as sm

from core.utils import ensure_dirs
from core.utils import read_excel_from_s3

In [2]:
# LOS FOLDERS y SCRIPS (notebook y .yaml) deben tener el mismo nombre
template_example = "aliar_sackoff/"
ROOT_DATA = Path(f"../data/{template_example}")
ROOT_IMAGEN =  Path(f"../images/{template_example}")

ensure_dirs(ROOT_DATA, ROOT_IMAGEN)
print(f"Carpetas verificadas/creadas:\n- {ROOT_DATA.resolve()}\n- {ROOT_IMAGEN.resolve()}")

Carpetas verificadas/creadas:
- /Users/juandavidrincon/Documents/galileo/data/aliar_sackoff
- /Users/juandavidrincon/Documents/galileo/images/aliar_sackoff


In [3]:
df = pd.read_csv("sackoff_aliar_roi.csv")
df = df[(df["planta_sackoff"] == "FAZENDA") & (df["despacho"]=="Si")]

df = df[df["mes_sackoff_fecha"].isin(['2025-02', '2025-03', '2025-04', '2025-05', '2025-06'])]

FileNotFoundError: [Errno 2] No such file or directory: 'sackoff_aliar_roi.csv'

In [771]:
df


,op_sackoff,orden_sackoff,material_sackoff,descripcion_sackoff,fecha_liberacion_raw,mes_sackoff_fecha,anno_sackoff,mes_sackoff_str2,cantidad_planificada,cantidad_entregada,...,code_642,diff,sackoff_pct,cons_cap,sackoff_cap,despacho,planta_sackoff,baches_sackoff,peso_agua,has_adiflow
0,20997.0,10013855,200102,GESTACION P CUARENTENA,01/06/2025,2025-06,2025,6/1/2025,"12,000.00","11,974.00",...,0,"-12,052.89",50.164,12013.452,-0.003300,Si,FAZENDA,6.0,111.15,Con Adiflow
1,20998.0,10013856,200099,GESTACION,01/06/2025,2025-06,2025,6/1/2025,"50,000.00","50,035.00",...,0,124.89,-0.250,49776.1244,0.005200,Si,FAZENDA,25.0,407.75,Con Adiflow
2,21004.0,10013857,200099,GESTACION,01/06/2025,2025-06,2025,6/1/2025,"20,000.00","20,200.00",...,0,191.26,-0.956,20008.733,0.009600,Si,FAZENDA,10.0,185.00,Con Adiflow
3,21006.0,10013858,200099,GESTACION,01/06/2025,2025-06,2025,6/1/2025,"12,000.00","11,536.00",...,0,-484.18,4.028,12020.171,-0.040300,Si,FAZENDA,6.0,110.95,Con Adiflow
4,20999.0,10013859,200119,FINALIZACION,01/06/2025,2025-06,2025,6/1/2025,"60,000.00","60,059.00",...,0,-63.04,0.105,60122.118,-0.001000,Si,FAZENDA,30.0,831.80,Con Adiflow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,NaN,10013847,200097,REEMPLAZOS S1B,45778,2025-05,2025,5/1/2025,"12,000.00",11635,...,0,-377.96,3.146,12012.962,-0.031463,Si,FAZENDA,NaN,110.90,Con Adiflow
1420,NaN,10013848,200099,GESTACION,45778,2025-05,2025,5/1/2025,100000,87254,...,0,-878.49,0.997,88132.473,-0.009968,Si,FAZENDA,NaN,813.60,Con Adiflow
1421,NaN,10013849,200103,LACTANCIA PRIMERIZA,45778,2025-05,2025,5/1/2025,164000,50146,...,0,-8.04,0.016,50154.04,-0.000160,Si,FAZENDA,NaN,461.85,Con Adiflow
1422,NaN,10013851,200118,INICIACION P INMUNIDAD,45778,2025-05,2025,5/1/2025,"26,000.00",21628,...,0,-4385.78,16.859,26013.7622,-0.168594,Si,FAZENDA,NaN,240.65,Con Adiflow


In [772]:

df[['cantidad_entregada', 'code_101', 'code_102', 'cons_cap']]

,cantidad_entregada,code_101,code_102,cons_cap
0,"11,974.00","11,974.00",0,12013.452
1,"50,035.00","50,035.00",0,49776.1244
2,"20,200.00","20,200.00",0,20008.733
3,"11,536.00","11,536.00",0,12020.171
4,"60,059.00","60,059.00",0,60122.118
...,...,...,...,...
1419,11635,11635,0,12012.962
1420,87254,87254,0,88132.473
1421,50146,50146,0,50154.04
1422,21628,21628,0,26013.7622


In [773]:
cols__num = ['cantidad_entregada', 'code_101', 'code_102','cons_cap']
for col in cols__num:
    # 1) Asegurarnos de trabajar sobre strings
    s = df[col].astype(str).str.strip()
    s = s.str.replace(',',    '', regex=False)
    df[col] = pd.to_numeric(s, errors='coerce')



In [774]:
df[['cantidad_entregada', 'code_101', 'code_102','cons_cap']]

,cantidad_entregada,code_101,code_102,cons_cap
0,11974.0,11974.0,0.0,12013.4520
1,50035.0,50035.0,0.0,49776.1244
2,20200.0,20200.0,0.0,20008.7330
3,11536.0,11536.0,0.0,12020.1710
4,60059.0,60059.0,0.0,60122.1180
...,...,...,...,...
1419,11635.0,11635.0,0.0,12012.9620
1420,87254.0,87254.0,0.0,88132.4730
1421,50146.0,50146.0,0.0,50154.0400
1422,21628.0,21628.0,0.0,26013.7622


In [775]:
df["peso_agua_form"] = df["cons_cap"]/1000 * 9.25
df["peso_agua_form"] = np.where(df["has_adiflow"] =='Sin Adiflow', 0, df["peso_agua_form"])

df["peso_adiflow_form"] = df["cons_cap"]/1000 * 0.75


df["peso_adiflow_form"] = np.where(df["has_adiflow"] == 'Sin Adiflow', 0, df["peso_adiflow_form"])


valor_general_dieta = 1_700_000 # Costo Promedio Dieta
valor_adiflow = 7_200 # Costo tonelada Adiflow

df["costo_dieta"] = df["cons_cap"]*valor_general_dieta
df["costo_adiflow"] = df["peso_adiflow_form"]*valor_adiflow

In [776]:
df["peso_adiflow_form"].sum()

24595.037911500003

In [777]:
df["fecha_liberacion_raw"].min()

'01/06/2025'

In [778]:
df["mes_sackoff_fecha"].value_counts()

2025-03    157
2025-04    157
2025-02    156
2025-05    142
2025-06    107
Name: mes_sackoff_fecha, dtype: int64

In [779]:
df["peso_adiflow_form"].sum()

24595.037911500003

In [780]:
35317

35317

In [781]:
cols = ['code_101', 'code_102', 'cons_cap']
for col in cols:
    df[col] = df[col]/1000

In [782]:
cols_group = ["mes_sackoff_fecha"]
#df1 = df[~df["orden_sackoff"].isin([10013921, 10014001, 10014079, 10014077])]
df1 = df[df["peso_agua"].notnull()] # "mes_sackoff_fecha"
df_group = df1.groupby(cols_group+["has_adiflow"]).agg(
    sum_code_101 = ("code_101", "sum"),
    sum_code_102 = ("code_102", "sum"),
    sum_cons_cap = ("cons_cap", "sum"),
    kilos_adiflow = ("peso_adiflow_form", "sum"),
)
df_group["diff_ton"] = df_group["sum_code_101"] - df_group["sum_cons_cap"] - df_group["sum_code_102"]
df_group["sack_off"]  = round(df_group["diff_ton"]/df_group["sum_cons_cap"]*100,2)

df_group = df_group.reset_index()


df_group_pivot = df_group.pivot(
    index='mes_sackoff_fecha',
    columns="has_adiflow", values=["sack_off", "sum_cons_cap", 'kilos_adiflow']).reset_index()
df_group_pivot.columns = ['mes_sackoff_fecha', 'sackoff_con', 'sackoff_sin', 'producido_con', 'producido_sin', 'kilos_con', 'kilos_sin']

for col in ['sackoff_con', 'sackoff_sin', 'producido_con', 'producido_sin', 'kilos_con', 'kilos_sin']:
    df_group_pivot[col] = pd.to_numeric(df_group_pivot[col], errors='coerce')

# --- 3) Calcula la diferencia de % sackoff ---
df_group_pivot['diferencia_sackoff'] = (
    df_group_pivot['sackoff_con'] - df_group_pivot['sackoff_sin']
)

# --- 4) Ahora sí multiplica por la columna producido_con ---
# (si diferencia_sackoff está en %, divides por 100 para tener el factor)
df_group_pivot['toneladas_recuperadas'] = (
    df_group_pivot['diferencia_sackoff']
    * df_group_pivot['producido_con']
    / 100
)

df_group_pivot["costo_recuperadas"] = df_group_pivot['producido_con']*valor_adiflow
df_group_pivot["valor_recuperadas"] = df_group_pivot['toneladas_recuperadas']*valor_general_dieta
df_group_pivot["roi"] = df_group_pivot["valor_recuperadas"]/df_group_pivot["costo_recuperadas"]
df_group_pivot


,mes_sackoff_fecha,sackoff_con,sackoff_sin,producido_con,producido_sin,kilos_con,kilos_sin,diferencia_sackoff,toneladas_recuperadas,costo_recuperadas,valor_recuperadas,roi
0,2025-02,-0.14,-1.61,6367.887417,3028.653047,4775.915562,0.0,1.47,93.607945,4.584879e+07,1.591335e+08,3.470833
1,2025-03,-0.83,-1.63,7212.666429,940.790431,5409.499822,0.0,0.80,57.701331,5.193120e+07,9.809226e+07,1.888889
2,2025-04,-1.72,-1.88,4548.571084,5266.442767,3411.428313,0.0,0.16,7.277714,3.274971e+07,1.237211e+07,0.377778
3,2025-05,-1.04,-2.63,8690.579612,903.781419,6517.934709,0.0,1.59,138.180216,6.257217e+07,2.349064e+08,3.754167
4,2025-06,-0.93,-1.30,5973.679341,994.160327,4480.259506,0.0,0.37,22.102614,4.301049e+07,3.757444e+07,0.873611


In [783]:
# ROI GLOBAL

#df1 = df[~df["orden_sackoff"].isin([10013921, 10014001, 10014079, 10014077])]
df1 = df[df["peso_agua"].notnull()] # "mes_sackoff_fecha"
df_group = df1.groupby(["has_adiflow"]).agg(
    sum_code_101 = ("code_101", "sum"),
    sum_code_102 = ("code_102", "sum"),
    sum_cons_cap = ("cons_cap", "sum"),
)
df_group["diff"] = df_group["sum_code_101"] - df_group["sum_cons_cap"] - df_group["sum_code_102"]
df_group["diff_toneladas"] = round(df_group["diff"]/1000, 2)
df_group["sack_off"]  = round(df_group["diff"]/df_group["sum_cons_cap"]*100,2)

df_group = df_group.reset_index()

diff_sackoff = -df_group.loc[df_group["has_adiflow"]=="Con Adiflow", "sack_off"][0] +df_group.loc[df_group["has_adiflow"]=="Sin Adiflow", "sack_off"][1]

toneladas_recuperadas = -diff_sackoff*df_group.loc[df_group["has_adiflow"]=="Con Adiflow", "sum_cons_cap"][0]/100
costo_toneladas_recuperadas = df_group.loc[df_group["has_adiflow"]=="Con Adiflow", "sum_cons_cap"][0]*valor_adiflow
valor_toneladas_recuperadas = toneladas_recuperadas*valor_general_dieta


roi = (valor_toneladas_recuperadas-costo_toneladas_recuperadas)/costo_toneladas_recuperadas
roi

1.1486111111111112

In [784]:
import pandas as pd
from typing import Optional

def calcular_roi_adiflow(
    df: pd.DataFrame,
    valor_adiflow: float,
    valor_general_dieta: float,
    group_col: Optional[str] = None,
    etiqueta_con: str = "Con Adiflow",
    etiqueta_sin: str = "Sin Adiflow"
) -> pd.DataFrame:
    """
    Calcula el ROI comparando lotes con/sin adiflow.
    - Si group_col es None o "", devuelve un DataFrame de una fila con el ROI global.
    - Si group_col es un nombre de columna, devuelve un DataFrame con una fila por cada valor de group_col.
    """
    # 1) Filtrar solo filas válidas
    df1 = df[df["peso_agua"].notnull()]

    # 2) Definir claves de agrupación
    if group_col:
        keys = [group_col, "has_adiflow"]
    else:
        keys = ["has_adiflow"]

    # 3) Agregación básica
    agg = df1.groupby(keys).agg(
        sum_code_101=("code_101", "sum"),
        sum_code_102=("code_102", "sum"),
        sum_cons_cap=("cons_cap", "sum"),
    )

    # 4) Calcular diff y % sack_off
    agg = agg.assign(
        diff=lambda d: d["sum_code_101"] - d["sum_cons_cap"] - d["sum_code_102"],
        sack_off=lambda d: (d["diff"] / d["sum_cons_cap"] * 100).round(2),
    )

    # 5A) Caso general (sin group_col)
    if not group_col:
        dfg = agg.reset_index()
        # Extraer valores Con y Sin
        sac_con = float(dfg.loc[dfg["has_adiflow"] == etiqueta_con, "sack_off"])
        sac_sin = float(dfg.loc[dfg["has_adiflow"] == etiqueta_sin, "sack_off"])
        cap_con = float(dfg.loc[dfg["has_adiflow"] == etiqueta_con, "sum_cons_cap"])

        diff_sack_off = sac_sin - sac_con
        toneladas_recup = - diff_sack_off * cap_con / 100
        costo_ton_recup = cap_con * valor_adiflow
        valor_ton_recup = toneladas_recup * valor_general_dieta
        roi = (valor_ton_recup - costo_ton_recup) / costo_ton_recup

        return pd.DataFrame([{
            "diff_sack_off": diff_sack_off,
            "toneladas_recuperadas": toneladas_recup,
            "costo_toneladas_recuperadas": costo_ton_recup,
            "valor_toneladas_recuperadas": valor_ton_recup,
            "ROI": roi
        }])

    # 5B) Caso por grupos
    # Pivot para separar Con/Sin por cada valor de group_col
    piv = (
        agg.reset_index()
           .pivot(index=group_col, columns="has_adiflow", values=["sack_off", "sum_cons_cap"])
    )
    # Aplanar columnas
    piv.columns = [
        f"{metric}_{'con' if lvl==etiqueta_con else 'sin'}"
        for metric, lvl in piv.columns
    ]

    # Cálculos
    piv.fillna(0, inplace=True)

    piv["diff_sack_off"] = piv[f"sack_off_sin"] - piv[f"sack_off_con"]
    piv["toneladas_recuperadas"] = - piv["diff_sack_off"] * piv["sum_cons_cap_con"] / 100
    piv["costo_toneladas_recuperadas"] = piv["sum_cons_cap_con"] * valor_adiflow
    piv["valor_toneladas_recuperadas"] = piv["toneladas_recuperadas"] * valor_general_dieta
    piv["ROI"] = (
        piv["valor_toneladas_recuperadas"] - piv["costo_toneladas_recuperadas"]
    ) / piv["costo_toneladas_recuperadas"]

    return piv


In [785]:
df_roi_por_mes = calcular_roi_adiflow(
    df=df,
    group_col="descripcion_sackoff",
    valor_adiflow=valor_adiflow,
    valor_general_dieta=valor_general_dieta
)

In [786]:
df_roi_por_mes

,sack_off_con,sack_off_sin,sum_cons_cap_con,sum_cons_cap_sin,diff_sack_off,toneladas_recuperadas,costo_toneladas_recuperadas,valor_toneladas_recuperadas,ROI
descripcion_sackoff,,,,,,,,,
B. NOVILLAS LEVANTE,0.00,-1.20,0.000000,102.003000,-1.20,0.000000,0.000000e+00,0.000000e+00,NaN
B. NOVILLOS CEBA,0.00,-5.63,0.000000,360.029000,-5.63,0.000000,0.000000e+00,0.000000e+00,NaN
ENGORDE ESP P,-1.30,0.15,1470.000436,2.196683,1.45,-21.315006,1.058400e+07,-3.623551e+07,-4.423611
ENGORDE ESP VR,1.30,-1.16,495.436428,389.573629,-2.46,12.187736,3.567142e+06,2.071915e+07,4.808333
FINALIZACION,-0.97,0.30,3899.763625,358.852596,1.27,-49.526998,2.807830e+07,-8.419590e+07,-3.998611
FINALIZADOR VR,-0.65,-0.54,775.335547,515.326315,0.11,-0.852869,5.582416e+06,-1.449877e+06,-1.259722
GESTACION,-1.20,-1.33,3953.692999,916.863357,-0.13,5.139801,2.846659e+07,8.737662e+06,-0.693056
GESTACION CUARENTENA ESP,-5.65,0.00,40.068200,47.959595,5.65,-2.263853,2.884910e+05,-3.848551e+06,-14.340278
GESTACION ESP P,-0.32,-1.05,1386.367099,447.864534,-0.73,10.120480,9.981843e+06,1.720482e+07,0.723611


In [787]:
df_roi_por_mes = calcular_roi_adiflow(
    df=df,
    valor_adiflow=valor_adiflow,
    valor_general_dieta=valor_general_dieta
)

In [788]:
df_roi_por_mes

,diff_sack_off,toneladas_recuperadas,costo_toneladas_recuperadas,valor_toneladas_recuperadas,ROI
0,-0.91,298.419793,2.361124e+08,5.073136e+08,1.148611


In [789]:
import pandas as pd
from typing import List, Optional

def calcular_roi_adiflow(
    df: pd.DataFrame,
    valor_adiflow: float,
    valor_general_dieta: float,
    cols_group: Optional[List[str]] = None,
    etiqueta_con: str = "Con Adiflow",
    etiqueta_sin: str = "Sin Adiflow",
    filtro_validez: str = "peso_agua"
) -> pd.DataFrame:
    """
    Calcula ROI comparando lotes con/sin adiflow, tanto global como por grupos.

    Parámetros
    ----------
    df : pd.DataFrame
        Debe contener al menos las columnas:
        - has_adiflow
        - code_101, code_102, cons_cap
        - filtro_validez (e.g. "peso_agua")
        - peso_adiflow_form (para sumar kilos_adiflow)
    valor_adiflow : float
        Costo unitario de adiflow.
    valor_general_dieta : float
        Valor por tonelada recuperada.
    cols_group : List[str] or None
        Columnas adicionales para agrupar. Si vacío o None, calcula global.
    etiqueta_con / etiqueta_sin : str
        Valores de `has_adiflow` para distinguir los dos grupos.
    filtro_validez : str
        Nombre de la columna que usas para filtrar filas válidas (p.ej. no-null).

    Devuelve
    -------
    pd.DataFrame
        Si cols_group es None o [], un DataFrame de 1 fila con:
        diff_sackoff, toneladas_recuperadas, costo_recuperadas,
        valor_recuperadas, roi.

        Si cols_group tiene columnas, un DataFrame con una fila por
        combinación de valores de esas columnas y las mismas métricas.
    """
    # 1) Filtrar filas válidas
    df1 = df[df[filtro_validez].notnull()]
    if cols_group:
        keys = cols_group + ["has_adiflow"]
    else:
        keys = ["has_adiflow"]

    agg = df1.groupby(keys).agg(
        sum_code_101    = ("code_101", "sum"),
        sum_code_102    = ("code_102", "sum"),
        sum_cons_cap    = ("cons_cap", "sum"),
        kilos_adiflow   = ("peso_adiflow_form", "sum"),

    )

    # 3) Calcular diff_ton y % sack_off
    agg = agg.assign(
        diff_ton=lambda d: d["sum_code_101"] - d["sum_cons_cap"] - d["sum_code_102"],
        sack_off=lambda d: (d["diff_ton"] / d["sum_cons_cap"] * 100).round(2),
    )

    # --- Caso GLOBAL (sin agrupación adicional) ---
    if not cols_group:
        dfg = agg.reset_index()
        # extraer métricas
        sac_con = float(dfg.loc[dfg["has_adiflow"] == etiqueta_con, "sack_off"])
        sac_sin = float(dfg.loc[dfg["has_adiflow"] == etiqueta_sin, "sack_off"])
        cap_con = float(dfg.loc[dfg["has_adiflow"] == etiqueta_con, "sum_cons_cap"])
        # cálculos
        diff_sackoff = sac_sin - sac_con
        toneladas_rec = -diff_sackoff * cap_con / 100
        costo_rec   = cap_con * valor_adiflow
        valor_rec   = toneladas_rec * valor_general_dieta
        roi         = (valor_rec-costo_rec) / costo_rec
        kilos_adiflow = df1["peso_adiflow_form"].sum()

        return pd.DataFrame([{
            "sack_off_sin": sac_sin,
            "sack_off_con": sac_con,
            "diff_sackoff": diff_sackoff,
            "toneladas_producidad_con_adiflow": cap_con,
            "toneladas_recuperadas": toneladas_rec,
            "costo_recuperadas": costo_rec,
            "kilos_adiflow": kilos_adiflow,
            "valor_recuperadas": valor_rec,
            "roi": roi
        }])

    # --- Caso POR GRUPOS ---
    # 4) Pivotar para separar Con/Sin en columnas
    piv = (
        agg
        .reset_index()
        .pivot(
            index=cols_group,
            columns="has_adiflow",
            values=["sack_off", "sum_cons_cap", "kilos_adiflow"]
        )
    )
    # aplanar MultiIndex
    piv.columns = [
        f"{metric}_{'con' if lvl==etiqueta_con else 'sin'}"
        for metric, lvl in piv.columns
    ]
    piv = piv.reset_index()

    # 5) Asegurar numérico
    for c in piv.columns:
        piv[c] = pd.to_numeric(piv[c], errors="coerce")

    # 6) Cálculos finales
    piv["diferencia_sackoff"]       = piv["sack_off_con"] - piv["sack_off_sin"]
    piv["toneladas_recuperadas"]    = piv["diferencia_sackoff"] * piv["sum_cons_cap_con"] / 100
    piv["costo_recuperadas"]        = piv["sum_cons_cap_con"] * valor_adiflow
    piv["valor_recuperadas"]        = piv["toneladas_recuperadas"] * valor_general_dieta
    piv["roi"]                      = (piv["valor_recuperadas"]-piv["costo_recuperadas"]) / piv["costo_recuperadas"]

    return piv

# ---------------- Ejemplos de uso ----------------



In [4]:
-0.54+0.98

0.43999999999999995

In [790]:
# 1) ROI global
df_roi_global = calcular_roi_adiflow(
    df,
    valor_general_dieta = 1_700_000,# Costo Promedio Dieta
    valor_adiflow = 5_400, # Costo tonelada Adiflow
    cols_group=[]
)
df_roi_global


,sack_off_sin,sack_off_con,diff_sackoff,toneladas_producidad_con_adiflow,toneladas_recuperadas,costo_recuperadas,kilos_adiflow,valor_recuperadas,roi
0,-1.8,-0.89,-0.91,32793.383882,298.419793,1.770843e+08,24595.037911,5.073136e+08,1.864815


In [791]:

# 2) ROI por mes_sackoff_fecha
df_roi_por_mes = calcular_roi_adiflow(
    df,
    valor_adiflow=0.75,
    valor_general_dieta=200.0,
    cols_group=["mes_sackoff_fecha"]
)
print(df_roi_por_mes)


   mes_sackoff_fecha  sack_off_con  sack_off_sin  sum_cons_cap_con  \
0                NaN         -0.14         -1.61       6367.887417   
1                NaN         -0.83         -1.63       7212.666429   
2                NaN         -1.72         -1.88       4548.571084   
3                NaN         -1.04         -2.63       8690.579612   
4                NaN         -0.93         -1.30       5973.679341   

   sum_cons_cap_sin  kilos_adiflow_con  kilos_adiflow_sin  diferencia_sackoff  \
0       3028.653047        4775.915562                0.0                1.47   
1        940.790431        5409.499822                0.0                0.80   
2       5266.442767        3411.428313                0.0                0.16   
3        903.781419        6517.934709                0.0                1.59   
4        994.160327        4480.259506                0.0                0.37   

   toneladas_recuperadas  costo_recuperadas  valor_recuperadas       roi  
0              93

In [797]:
import pandas as pd
from typing import List, Optional

def calcular_roi_adiflow(
    df: pd.DataFrame,
    valor_adiflow: float,
    valor_general_dieta: float,
    cols_group: Optional[List[str]] = None,
    etiqueta_con: str = "Con Adiflow",
    etiqueta_sin: str = "Sin Adiflow",
    filtro_validez: str = "peso_agua",
    col_peso_adiflow: str = "peso_adiflow_form"
) -> pd.DataFrame:
    """
    Calcula el ROI comparando lotes con y sin Adiflow.

    Salida en ambos casos (global o por grupo) con columnas renombradas:
      - Sackoff Sin Adiflow
      - Sackoff Con Adiflow
      - Diferencia Sackoff
      - Toneladas Producidas Con Adiflow
      - Toneladas Recuperadas
      - Costo Toneladas Recuperasas
      - Kilos de Adiflow
      - Valor Recuperasas
      - ROI
    """
    # Filtrar filas válidas
    df_valid = df[df[filtro_validez].notna()].copy()

    # Definir claves de agrupación
    if cols_group:
        group_keys = cols_group + ["has_adiflow"]
    else:
        group_keys = ["has_adiflow"]

    # Agregación inicial
    resumen = (
        df_valid
        .groupby(group_keys)
        .agg(
            sum_code_101=("code_101", "sum"),
            sum_code_102=("code_102", "sum"),
            sum_cons_cap=("cons_cap", "sum"),
            kilos_adiflow=(col_peso_adiflow, "sum"),
        )
        .assign(
            diff_ton=lambda d: d["sum_code_101"] - d["sum_cons_cap"] - d["sum_code_102"],
            sack_off=lambda d: (d["diff_ton"] / d["sum_cons_cap"] * 100).round(2),
        )
    )

    # Nombre de salida
    rename_map = {
        'sack_off_sin': 'Sackoff Sin Adiflow',
        'sack_off_con': 'Sackoff Con Adiflow',
        'diff_sackoff': 'Diferencia Sackoff',
        'toneladas_producidad_con_adiflow': 'Toneladas Producidas Con Adiflow',
        'toneladas_recuperadas': 'Toneladas Recuperadas',
        'costo_recuperadas': 'Costo Toneladas Recuperadas ($)',
        'kilos_adiflow': 'Kilos de Adiflow',
        'valor_recuperadas': 'Valor Toneladas Recuperadas ($)',
        'roi': 'ROI'
    }

    # Calcular global
    if not cols_group:
        df_tot = resumen.reset_index()
        def get_val(df_, label, col):
            return float(df_.loc[df_["has_adiflow"] == label, col].iloc[0])

        sac_con = get_val(df_tot, etiqueta_con, "sack_off")
        sac_sin = get_val(df_tot, etiqueta_sin, "sack_off")
        cap_con = get_val(df_tot, etiqueta_con, "sum_cons_cap")
        kilos_con = get_val(df_tot, etiqueta_con, "kilos_adiflow")

        diff = sac_con - sac_sin
        ton_prod = cap_con
        ton_rec = diff * cap_con / 100
        costo = cap_con * valor_adiflow
        valor = ton_rec * valor_general_dieta
        roi_val = (valor - costo) / costo

        df_res = pd.DataFrame([{
            'sack_off_sin': sac_sin,
            'sack_off_con': sac_con,
            'diff_sackoff': diff,
            'toneladas_producidad_con_adiflow': ton_prod,
            'toneladas_recuperadas': ton_rec,
            'kilos_adiflow': kilos_con,
            'costo_recuperadas': costo,
            'valor_recuperadas': valor,
            'roi': roi_val
        }])
        # Renombrar y ordenar
        return df_res.rename(columns=rename_map)[list(rename_map.values())].sort_values(by='ROI', ascending=False).round(2)

    # Calcular por grupos
    df_sum = resumen.reset_index()
    piv = df_sum.pivot(
        index=cols_group,
        columns="has_adiflow",
        values=["sack_off", "sum_cons_cap", "kilos_adiflow"]
    )
    piv.columns = [
        f"{metric}_{'con' if lvl==etiqueta_con else 'sin'}"
        for metric, lvl in piv.columns
    ]
    df_out = piv.reset_index()
    # Asegurar numérico en métricas
    metrics = [c for c in df_out.columns if c not in (cols_group or [])]
    for c in metrics:
        df_out[c] = pd.to_numeric(df_out[c], errors="coerce")

    # Calcular columnas base
    df_out['sack_off_sin'] = df_out['sack_off_sin']
    df_out['sack_off_con'] = df_out['sack_off_con']
    df_out['diff_sackoff'] = df_out['sack_off_con'] - df_out['sack_off_sin']
    df_out['toneladas_producidad_con_adiflow'] = df_out['sum_cons_cap_con']
    df_out['toneladas_recuperadas'] = df_out['diff_sackoff'] * df_out['sum_cons_cap_con'] / 100
    df_out['costo_recuperadas'] = df_out['sum_cons_cap_con'] * valor_adiflow
    df_out['valor_recuperadas'] = df_out['toneladas_recuperadas'] * valor_general_dieta
    df_out['roi'] = (df_out['valor_recuperadas'] - df_out['costo_recuperadas']) / df_out['costo_recuperadas']
    df_out['kilos_adiflow'] = df_out['kilos_adiflow_con']

    # Seleccionar y renombrar
    cols_final = (cols_group or []) + [
        'sack_off_sin', 'sack_off_con', 'diff_sackoff',
        'toneladas_producidad_con_adiflow', 'toneladas_recuperadas','kilos_adiflow',
        'costo_recuperadas', 'valor_recuperadas', 'roi'
    ]
    df_final = df_out[cols_final].rename(columns=rename_map)
    return df_final.sort_values(by='ROI', ascending=False).round(2)

# Ejemplos
# global: calcular_roi_adiflow(df, 0.75, 200.0)
# por mes: calcular_roi_adiflow(df, 0.75, 200.0, cols_group=["mes_sackoff_fecha"])


In [798]:
# 1) ROI global
df_roi_global = calcular_roi_adiflow(
    df,
    valor_general_dieta = 1_700_000,# Costo Promedio Dieta
    valor_adiflow = 5_400, # Costo tonelada Adiflow
    cols_group=[]
)
global_roi = df_roi_global
global_roi.to_csv(f"{ROOT_DATA}/GLOBAL_ROI.csv", index=False)
global_roi

,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas,Costo Toneladas Recuperadas ($),Kilos de Adiflow,Valor Toneladas Recuperadas ($),ROI
0,-1.8,-0.89,0.91,32793.38,298.42,1.770843e+08,24595.04,5.073136e+08,1.86


In [813]:

# 2) ROI por mes_sackoff_fecha
df_roi_por_mes = calcular_roi_adiflow(
    df,
     valor_general_dieta = 1_700_000,# Costo Promedio Dieta
    valor_adiflow = 5_400, # Costo tonelada Adiflow
    cols_group=["mes_sackoff_fecha"]
)
df_roi_por_mes

,mes_sackoff_fecha,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas,Kilos de Adiflow,Costo Toneladas Recuperadas ($),Valor Toneladas Recuperadas ($),ROI
3,2025-05,-2.63,-1.04,1.59,8690.58,138.18,6517.93,46929129.90,2.349064e+08,4.01
0,2025-02,-1.61,-0.14,1.47,6367.89,93.61,4775.92,34386592.05,1.591335e+08,3.63
1,2025-03,-1.63,-0.83,0.80,7212.67,57.70,5409.50,38948398.72,9.809226e+07,1.52
4,2025-06,-1.30,-0.93,0.37,5973.68,22.10,4480.26,32257868.44,3.757444e+07,0.16
2,2025-04,-1.88,-1.72,0.16,4548.57,7.28,3411.43,24562283.85,1.237211e+07,-0.50


In [800]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Ensure you have kaleido installed for static image export:
# pip install kaleido

def plot_horizontal_bar_chart_plotly(
    df,
    category_col,
    value_col,
    title="",
    bar_color="#1A494C",
    filename=None,
    width=1000, # New: Width for the plot
    height=700, # New: Height for the plot
    save_as_image=False # New: Option to save as static image (e.g., PNG)
):
    """
    Generates an interactive horizontal bar chart using Plotly and can save it to an HTML file.
    Axes and text labels are set to black for better contrast.
    Bars are sorted from lowest value to highest (ascending for horizontal bars often looks better).
    The plot's width and height can be specified.

    Args:
        df (pd.DataFrame): The input DataFrame.
        category_col (str): The name of the column to use for categories (y-axis labels).
        value_col (str): The name of the column to use for values (x-axis).
        title (str, optional): The title of the plot. Defaults to "".
        bar_color (str, optional): The color of the bars in hexadecimal or RGB format. Defaults to "#1A494C".
        filename (str, optional): The base name for the output file (e.g., "my_plot").
                                  If provided, it will save as "my_plot.html".
                                  If save_as_image is True, it will also save as "my_plot.png".
        width (int, optional): The width of the plot in pixels. Defaults to 1000.
        height (int, optional): The height of the plot in pixels. Defaults to 700.
        save_as_image (bool, optional): If True, also saves the plot as a static image (PNG).
                                        Requires 'kaleido' engine (pip install kaleido). Defaults to False.
    """
    # 1. Ordenar el DataFrame por la columna de valor (ROI) de forma ascendente para barras horizontales
    # Generalmente, para barras horizontales, se ordena de menor a mayor para que el "más alto" quede arriba.
    df_sorted = df.sort_values(by=value_col, ascending=True)

    # 2. Crear el gráfico de barras horizontal con Plotly Express
    fig = px.bar(
        df_sorted,
        x=value_col,
        y=category_col,
        orientation='h', # 'h' para barras horizontales
        title=title,
        text=value_col # Mostrar el valor en las barras
    )

    # 3. Personalizar la apariencia de las barras y el texto
    fig.update_traces(
        marker_color=bar_color, # Color específico de las barras (verde oscuro)
        texttemplate='%{x:.2f}', # Formato del texto sobre las barras (2 decimales)
        textposition='outside', # Posición del texto fuera de las barras
        textfont_color='black' # Color del texto sobre las barras (negro para contraste)
    )

    # 4. Personalizar el diseño general del gráfico (fondo, ejes, títulos, y dimensiones)
    fig.update_layout(
        # Fondos transparentes para el área del gráfico y el área de trazado
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',

        # Color del título principal del gráfico (negro)
        title_font_color='black',

        # Configuración del eje X (valores)
        xaxis=dict(
            title=value_col, # Título del eje X
            title_font_color='black', # Color del título del eje X
            tickfont_color='black',   # Color de las etiquetas (valores numéricos) del eje X
            gridcolor='lightgray',    # Color de las líneas de la cuadrícula
            gridwidth=1,
            griddash='dash',          # Estilo de la cuadrícula (discontinua)
            showline=False            # Quitar la línea del eje X
        ),

        # Configuración del eje Y (categorías)
        yaxis=dict(
            title=category_col,       # Título del eje Y
            title_font_color='black', # Color del título del eje Y
            tickfont_color='black',   # Color de las etiquetas (nombres de categorías) del eje Y
            categoryorder='array',    # Asegura que las categorías sigan el orden del array
            categoryarray=df_sorted[category_col].tolist(), # Array con las categorías ya ordenadas
            showgrid=False,           # Quitar las líneas de la cuadrícula del eje Y
            showline=False            # Quitar la línea del eje Y
        ),

        # Establecer el ancho y alto de la figura
        width=width,
        height=height
    )

    # 5. Guardar el gráfico o mostrarlo
    if filename:
        html_filename = f"{filename}"
        fig.write_html(html_filename, auto_open=False)
        print(f"Gráfico interactivo guardado como '{html_filename}'.")

        if save_as_image:
            image_filename = f"{filename}.png"
            try:
                fig.write_image(image_filename, width=width, height=height)
                print(f"Gráfico estático guardado como '{image_filename}'.")
            except ValueError as e:
                print(f"Error al guardar la imagen estática: {e}")
                print("Asegúrate de tener el motor 'kaleido' instalado: pip install kaleido")

    fig.show() # Siempre muestra el gráfico en el entorno de desarrollo


In [801]:
# 2) ROI por mes_sackoff_fecha
df_roi_por_dieta = calcular_roi_adiflow(
    df,
     valor_general_dieta = 1_700_000,# Costo Promedio Dieta
    valor_adiflow = 5_400, # Costo tonelada Adiflow
    cols_group=["descripcion_sackoff"]
)
df_roi_por_dieta = df_roi_por_dieta.dropna()

In [802]:
plot_horizontal_bar_chart_plotly(
    df_roi_por_dieta,
    category_col='descripcion_sackoff',
    value_col='ROI',
    title='ROI por Dieta',
    bar_color='#1A494C', # Verde oscuro
    width=1000,
    height=500,
    save_as_image=True,
    filename=f'{ROOT_IMAGEN}/ROI_by_product.html'
)

Gráfico interactivo guardado como '../images/aliar_sackoff/ROI_by_product.html'.
Gráfico estático guardado como '../images/aliar_sackoff/ROI_by_product.html.png'.


In [803]:
plot_horizontal_bar_chart_plotly(
    df_roi_por_dieta,
    category_col='descripcion_sackoff',
    value_col='Toneladas Recuperadas',
    title='Toneladas recuperadas por Dieta',
    bar_color='#1A494C', # Verde oscuro
    width=1000,
    height=500,
    save_as_image=True,
    filename=f'{ROOT_IMAGEN}/ton_recuperadas_by_product.html'
)

Gráfico interactivo guardado como '../images/aliar_sackoff/ton_recuperadas_by_product.html'.
Gráfico estático guardado como '../images/aliar_sackoff/ton_recuperadas_by_product.html.png'.


In [804]:
df_roi_por_dieta.to_csv(f"{ROOT_DATA}/datos_roi_por_dieta.csv", index=False)

In [815]:
df_roi_por_dieta.to_excel(f"{ROOT_DATA}/datos_roi_por_dieta.xlsx", index=False)

In [805]:
import plotly.graph_objects as go
import pandas as pd
import os # Importar para manejar rutas de archivo

def plot_comparison_bars(
    df: pd.DataFrame,
    category_col: str,
    value_col1: str,
    value_col2: str,
    name_col1: str,
    name_col2: str,
    color1: str = '#94AF92',  # Default Light Green for the first bar
    color2: str = '#1A494C',  # Default Dark Green for the second bar
    title: str = 'Comparación de Valores',
    xaxis_title: str = 'Valor',
    yaxis_title: str = 'Categoría',
    show_annotations: bool = True,
    annotation_format: str = 'Diferencia: {:.2f}',
    annotation_font_size: int = 10,
    transparent_background: bool = True,
    order_by_change: bool = True,
    ascending_order: bool = True, # Orden ascendente (si order_by_change es True y sort_positive_first es False)
    sort_positive_first: bool = False, # NUEVO: Ordenar los cambios positivos primero, luego los negativos
    width: int = 1000,
    height: int = 700,
    save_path: str = None
) -> go.Figure:
    """
    Genera un gráfico de barras de comparación horizontal con dos series de datos.

    Args:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        category_col (str): Nombre de la columna para las categorías (Eje Y).
        value_col1 (str): Nombre de la primera columna de valores (para la primera barra).
        value_col2 (str): Nombre de la segunda columna de valores (para la segunda barra).
        name_col1 (str): Nombre a mostrar en la leyenda para la primera serie de barras.
        name_col2 (str): Nombre a mostrar en la leyenda para la segunda serie de barras.
        color1 (str, optional): Color de la primera serie de barras. Por defecto 'light green'.
        color2 (str, optional): Color de la segunda serie de barras. Por defecto 'dark green'.
        title (str, optional): Título principal del gráfico. Por defecto 'Comparación de Valores'.
        xaxis_title (str, optional): Título del eje X. Por defecto 'Valor'.
        yaxis_title (str, optional): Título del eje Y. Por defecto 'Categoría'.
        show_annotations (bool, optional): Si se deben mostrar las anotaciones de diferencia. Por defecto True.
        annotation_format (str, optional): Formato de la cadena para la anotación. Utiliza sintaxis de f-string.
                                            Por defecto 'Diferencia: {:.2f}'.
        annotation_font_size (int, optional): Tamaño de la fuente de las anotaciones. Por defecto 10.
        transparent_background (bool, optional): Si el fondo del gráfico debe ser transparente. Por defecto True.
        order_by_change (bool, optional): Si se debe ordenar el gráfico por la diferencia entre value_col2 y value_col1.
                                          Por defecto True.
        ascending_order (bool, optional): Si el orden es ascendente (True) o descendente (False) cuando se ordena
                                          por el cambio (solo si sort_positive_first es False). Por defecto False.
        sort_positive_first (bool, optional): Si se deben ordenar los cambios positivos primero (descendente) y luego
                                              los cambios negativos (descendente). Anula 'ascending_order' si es True.
                                              Por defecto False.
        width (int, optional): Ancho de la figura en píxeles. Por defecto 1000.
        height (int, optional): Alto de la figura en píxeles. Por defecto 700.
        save_path (str, optional): Ruta completa del archivo para guardar la imagen (ej. 'grafico.png').
                                   Si se proporciona, el gráfico se guardará en esa ruta. Requiere 'kaleido'.

    Returns:
        go.Figure: El objeto Plotly Figure.
    """

    # Create a copy to avoid modifying the original DataFrame
    df_plot = df.copy()

    # Calculate the difference for annotations and potential sorting
    df_plot['__temp_diff__'] = df_plot[value_col2] - df_plot[value_col1]

    # Order the DataFrame based on the specified logic
    if order_by_change:
        if sort_positive_first:
            # Create a boolean column: True for positive changes, False for non-positive
            df_plot['__is_positive__'] = df_plot['__temp_diff__'] > 0
            # Sort by:
            # 1. '__is_positive__' descending (True comes before False)
            # 2. '__temp_diff__' descending (largest positive first, then largest negative/closest to zero first)
            df_plot = df_plot.sort_values(
                by=['__is_positive__', '__temp_diff__'],
                ascending=[False, False]
            )
            df_plot.drop(columns=['__is_positive__'], inplace=True) # Clean up temp column
        else:
            # Standard sorting as before
            df_plot = df_plot.sort_values(by='__temp_diff__', ascending=ascending_order)

    # Create the figure
    fig = go.Figure()

    # Add bars for the first value column
    fig.add_trace(go.Bar(
        y=df_plot[category_col], # Use the sorted DataFrame
        x=df_plot[value_col1],
        name=name_col1,
        orientation='h',
        marker_color=color1
    ))

    # Add bars for the second value column
    fig.add_trace(go.Bar(
        y=df_plot[category_col], # Use the sorted DataFrame
        x=df_plot[value_col2],
        name=name_col2,
        orientation='h',
        marker_color=color2
    ))

    # Update layout
    layout_params = {
        'title_text': title,
        'xaxis_title': xaxis_title,
        'yaxis_title': yaxis_title,
        'barmode': 'group',
        'legend_title_text': 'Series',
        'margin': dict(l=max(100, 15 * df_plot[category_col].astype(str).str.len().max()), r=50, t=50, b=50),
        'font': dict(
            family="Arial, sans-serif",
            size=12,
            color="Black"
        ),
        'title_x': 0.5,
        'width': width,
        'height': height
    }

    # Apply transparent background if requested
    if transparent_background:
        layout_params['paper_bgcolor'] = 'rgba(0,0,0,0)'
        layout_params['plot_bgcolor'] = 'rgba(0,0,0,0)'

    fig.update_layout(**layout_params)

    # Add annotations if requested
    if show_annotations:
        for i, row in df_plot.iterrows():
            diff_value = row['__temp_diff__']
            annotation_text = annotation_format.format(diff_value)

            # Determine arrow direction
            if diff_value < 0:
                annotation_text += " \u25BC" # Down arrow
            elif diff_value > 0:
                annotation_text += " \u25B2" # Up arrow

            x_annotation_pos = row[value_col2]
            x_range = df_plot[[value_col1, value_col2]].values.flatten()
            max_abs_val = max(abs(x_range.min()), abs(x_range.max())) if x_range.size > 0 else 1

            offset_factor = 0.05
            dynamic_offset = offset_factor * max_abs_val

            xanchor_setting = 'left'

            if row[value_col2] < 0:
                x_annotation_pos -= dynamic_offset
                xanchor_setting = 'right'
            else:
                x_annotation_pos += dynamic_offset
                xanchor_setting = 'left'

            fig.add_annotation(
                x=x_annotation_pos,
                y=row[category_col],
                text=annotation_text,
                showarrow=False,
                font=dict(
                    size=annotation_font_size,
                    color="black"
                ),
                xanchor=xanchor_setting,
                yanchor='middle'
            )

    # Drop the temporary column before returning
    if '__temp_diff__' in df_plot.columns:
        df_plot.drop(columns=['__temp_diff__'], inplace=True)

    # Save the figure as an image if save_path is provided
    if save_path:
        try:
            output_dir = os.path.dirname(save_path)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir)

            fig.write_image(save_path, width=width, height=height, scale=2)
            print(f"Gráfico guardado como '{save_path}' con dimensiones {width}x{height} y escala 2.")
        except ValueError as e:
            print(f"Error al guardar la imagen estática: {e}")
            print("Asegúrate de tener el motor 'kaleido' instalado: pip install kaleido")
        except Exception as e:
            print(f"Ocurrió un error inesperado al guardar la imagen: {e}")

    return fig

#

In [806]:
# 2. Llamada a la función para generar el gráfico con tus datos y colores específicos
fig_roi = plot_comparison_bars(
    df=df_roi_por_dieta,
    category_col='descripcion_sackoff',
    value_col1='Sackoff Sin Adiflow',
    value_col2='Sackoff Con Adiflow',
    name_col1='Sackoff Sin Adiflow',
    name_col2='Sackoff Con Adiflow',
    color1='#94AF92',  # Verde claro para 'Sin Adiflow'
    color2='#1A494C',  # Verde oscuro para 'Con Adiflow'
    title='Comparación por Dieta (Sackoff)',
    xaxis_title='Sackoff(%)',
    yaxis_title='Dieta',
    width=1000, # Ancho deseado
    height=500, # Alto deseado
    annotation_format='{:.2f}' # Personalizar el texto de la anotación
)

fig_roi.show()
fig_roi.write_html(f'{ROOT_IMAGEN}/comparacion_sackoff.html', auto_open=False) # Guarda como HTML, no lo abre automáticamente
print(f"Gráfico interactivo guardado como f'{ROOT_IMAGEN}/comparacion_sackoff.html'.")


Gráfico interactivo guardado como f'../images/aliar_sackoff/comparacion_sackoff.html'.


In [807]:
df_roi_por_mes

,mes_sackoff_fecha,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas,Kilos de Adiflow,Costo Toneladas Recuperadas ($),Valor Toneladas Recuperadas ($),ROI
3,2025-05,-2.63,-1.04,1.59,8690.58,138.18,6517.93,46929129.90,2.349064e+08,4.01
0,2025-02,-1.61,-0.14,1.47,6367.89,93.61,4775.92,34386592.05,1.591335e+08,3.63
1,2025-03,-1.63,-0.83,0.80,7212.67,57.70,5409.50,38948398.72,9.809226e+07,1.52
4,2025-06,-1.30,-0.93,0.37,5973.68,22.10,4480.26,32257868.44,3.757444e+07,0.16
2,2025-04,-1.88,-1.72,0.16,4548.57,7.28,3411.43,24562283.85,1.237211e+07,-0.50


In [808]:
# 2. Llamada a la función para generar el gráfico con tus datos y colores específicos
fig_roi = plot_comparison_bars(
    df=df_roi_por_mes,
    category_col='mes_sackoff_fecha',
    value_col1='Sackoff Sin Adiflow',
    value_col2='Sackoff Con Adiflow',
    name_col1='Sackoff Sin Adiflow',
    name_col2='Sackoff Con Adiflow',
    color1='#94AF92',  # Verde claro para 'Sin Adiflow'
    color2='#1A494C',  # Verde oscuro para 'Con Adiflow'
    title='Comparación por Mes',
    xaxis_title='Sackoff(%)',
    yaxis_title='Mes',
    width=700, # Ancho deseado
    height=400, # Alto deseado
    annotation_format='{:.2f}' # Personalizar el texto de la anotación
)

fig_roi.show()
fig_roi.write_html(f'{ROOT_IMAGEN}/comparacion_mes.html', auto_open=False) # Guarda como HTML, no lo abre automáticamente
print(f"Gráfico interactivo guardado como f'{ROOT_IMAGEN}/comparacion_sackoff.html'.")


Gráfico interactivo guardado como f'../images/aliar_sackoff/comparacion_sackoff.html'.


In [809]:
df_roi_por_dieta

,descripcion_sackoff,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas,Kilos de Adiflow,Costo Toneladas Recuperadas ($),Valor Toneladas Recuperadas ($),ROI
11,LACTANCIA PRIMERIZA,-3.54,-0.42,3.12,4057.54,126.60,3043.15,21910706.64,2.152118e+08,8.82
24,REEMPLAZOS S1B,-4.79,-1.74,3.05,667.43,20.36,500.57,3604128.30,3.460631e+07,8.60
3,ENGORDE ESP VR,-1.16,1.30,2.46,495.44,12.19,371.58,2675356.71,2.071915e+07,6.74
21,PREINICIACION F1 P INMUNIDAD,-1.38,0.52,1.90,26.12,0.50,19.59,141063.49,8.437687e+05,4.98
18,LEVANTE VR,-2.31,-0.50,1.81,2495.38,45.17,1871.53,13475046.92,7.678281e+07,4.70
12,LACTANCIA PRIMERIZAS ESP,-2.70,-1.03,1.67,1342.60,22.42,1006.95,7250031.60,3.811637e+07,4.26
23,PRELACTANCIA H,-3.48,-2.26,1.22,34.05,0.42,25.54,183883.77,7.062499e+05,2.84
25,TERNERAS H,-7.97,-6.89,1.08,75.96,0.82,56.97,410209.65,1.394713e+06,2.40
10,INICIACION P INMUNIDAD,-2.08,-1.06,1.02,5957.12,60.76,4467.84,32168431.18,1.032964e+08,2.21
20,NOVILLONAS SUPLEMENTO,-2.43,-1.47,0.96,56.13,0.54,42.10,303114.47,9.160793e+05,2.02


In [810]:
import plotly.graph_objects as go
import pandas as pd
import os # Importar para manejar rutas de archivo

def plot_single_bar_chart(
    df: pd.DataFrame,
    category_col: str,
    value_col: str,
    title: str = 'Valores por Categoría',
    bar_color: str = '#1A494C',  # Color corporativo principal (Verde Oscuro)
    text_color: str = 'black',   # Color del texto sobre las barras
    xaxis_title: str = 'Valor',
    yaxis_title: str = 'Categoría',
    transparent_background: bool = True,
    order_descending: bool = True, # Ordenar de mayor a menor valor (para que el más grande esté arriba)
    width: int = 800,
    height: int = 500,
    save_path: str = None # Ruta completa para guardar la imagen (e.g., 'mi_grafico.png')
) -> go.Figure:
    """
    Genera un gráfico de barras horizontal simple con una columna categórica y una de valores.

    Args:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        category_col (str): Nombre de la columna para las categorías (Eje Y).
        value_col (str): Nombre de la columna de valores (Eje X).
        title (str, optional): Título principal del gráfico. Por defecto 'Valores por Categoría'.
        bar_color (str, optional): Color de las barras. Por defecto el verde oscuro corporativo ('#1A494C').
        text_color (str, optional): Color del texto que muestra el valor sobre las barras. Por defecto 'black'.
        xaxis_title (str, optional): Título del eje X. Por defecto 'Valor'.
        yaxis_title (str, optional): Título del eje Y. Por defecto 'Categoría'.
        transparent_background (bool, optional): Si el fondo del gráfico debe ser transparente. Por defecto True.
        order_descending (bool, optional): Si las barras deben ordenarse de mayor a menor valor (True) o viceversa (False).
                                           Por defecto True (mayor valor arriba).
        width (int, optional): Ancho de la figura en píxeles. Por defecto 1000.
        height (int, optional): Alto de la figura en píxeles. Por defecto 700.
        save_path (str, optional): Ruta completa del archivo para guardar la imagen (ej. 'grafico.png').
                                   Si se proporciona, el gráfico se guardará en esa ruta. Requiere 'kaleido'.

    Returns:
        go.Figure: El objeto Plotly Figure.
    """

    # Crear una copia para evitar modificar el DataFrame original
    df_plot = df.copy()

    # Ordenar el DataFrame según el valor para que las barras aparezcan ordenadas
    # Para barras horizontales, 'ascending=False' significa que el valor más grande estará arriba.
    df_plot = df_plot.sort_values(by=value_col, ascending=not order_descending)

    # Crear la figura
    fig = go.Figure()

    # Añadir la traza de barras
    fig.add_trace(go.Bar(
        y=df_plot[category_col],
        x=df_plot[value_col],
        orientation='h', # Barras horizontales
        marker_color=bar_color,
        text=df_plot[value_col], # Mostrar el valor en la barra
        texttemplate='%{x:.2f}', # Formato del texto (2 decimales)
        textposition='outside',  # Posición del texto (fuera de la barra)
        textfont=dict(color=text_color, size=8) # Estilo del texto
    ))

    # Configurar el layout del gráfico
    layout_params = {
        'title_text': title,
        'xaxis_title': xaxis_title,
        'yaxis_title': yaxis_title,
        'margin': dict(l=max(100, 15 * df_plot[category_col].astype(str).str.len().max()), r=100, t=50, b=50), # Margen dinámico
        'font': dict(
            family="Arial, sans-serif",
            size=12,
            color="Black"
        ),
        'title_x': 0.5, # Centrar título
        'width': width,
        'height': height,
        # Configuración de ejes para mejor visibilidad y sin líneas de cuadrícula innecesarias
        'xaxis': dict(
            title_font_color='black',
            tickfont_color='black',
            gridcolor='lightgray',
            gridwidth=1,
            griddash='dash',
            showline=False # Quitar la línea del eje
        ),
        'yaxis': dict(
            title_font_color='black',
            tickfont_color='black',
            categoryorder='array', # Mantener el orden del array (ya ordenado por sort_values)
            categoryarray=df_plot[category_col].tolist(),
            showgrid=False, # Quitar cuadrícula del eje Y
            showline=False  # Quitar la línea del eje
        )
    }

    # Aplicar fondos transparentes si se solicita
    if transparent_background:
        layout_params['paper_bgcolor'] = 'rgba(0,0,0,0)'
        layout_params['plot_bgcolor'] = 'rgba(0,0,0,0)'

    fig.update_layout(**layout_params)

    # Guardar la figura como imagen si se proporciona save_path
    if save_path:
        try:
            output_dir = os.path.dirname(save_path)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # scale=2 para mayor resolución
            fig.write_image(save_path, width=width, height=height, scale=2)
            #fig_roi.write_html(f'{ROOT_IMAGEN}/badd.html', auto_open=False) # Guarda como HTML, no lo abre automáticamente

            print(f"Gráfico guardado como '{save_path}' con dimensiones {width}x{height} y escala 2.")
        except ValueError as e:
            print(f"Error al guardar la imagen estática: {e}")
            print("Asegúrate de tener el motor 'kaleido' instalado: pip install kaleido")
        except Exception as e:
            print(f"Ocurrió un error inesperado al guardar la imagen: {e}")

    return fig



In [811]:

fig_roi_negativo = plot_single_bar_chart(
        df=df_roi_por_dieta,
        category_col='descripcion_sackoff',
        value_col='Toneladas Recuperadas',
        title='ROI por Proyecto',
        xaxis_title='ROI',
        yaxis_title='Dieta',
        bar_color='#1A494C',
        order_descending=False, # Mayor ROI (menos negativo) arriba
        width=900,
        height=550,
        save_path=f'{ROOT_IMAGEN}/toneladas_recuperadas.png'
    )
fig_roi_negativo.show()

Gráfico guardado como '../images/aliar_sackoff/toneladas_recuperadas.png' con dimensiones 900x550 y escala 2.


In [812]:
df_roi_por_dieta

,descripcion_sackoff,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas,Kilos de Adiflow,Costo Toneladas Recuperadas ($),Valor Toneladas Recuperadas ($),ROI
11,LACTANCIA PRIMERIZA,-3.54,-0.42,3.12,4057.54,126.60,3043.15,21910706.64,2.152118e+08,8.82
24,REEMPLAZOS S1B,-4.79,-1.74,3.05,667.43,20.36,500.57,3604128.30,3.460631e+07,8.60
3,ENGORDE ESP VR,-1.16,1.30,2.46,495.44,12.19,371.58,2675356.71,2.071915e+07,6.74
21,PREINICIACION F1 P INMUNIDAD,-1.38,0.52,1.90,26.12,0.50,19.59,141063.49,8.437687e+05,4.98
18,LEVANTE VR,-2.31,-0.50,1.81,2495.38,45.17,1871.53,13475046.92,7.678281e+07,4.70
12,LACTANCIA PRIMERIZAS ESP,-2.70,-1.03,1.67,1342.60,22.42,1006.95,7250031.60,3.811637e+07,4.26
23,PRELACTANCIA H,-3.48,-2.26,1.22,34.05,0.42,25.54,183883.77,7.062499e+05,2.84
25,TERNERAS H,-7.97,-6.89,1.08,75.96,0.82,56.97,410209.65,1.394713e+06,2.40
10,INICIACION P INMUNIDAD,-2.08,-1.06,1.02,5957.12,60.76,4467.84,32168431.18,1.032964e+08,2.21
20,NOVILLONAS SUPLEMENTO,-2.43,-1.47,0.96,56.13,0.54,42.10,303114.47,9.160793e+05,2.02
